In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Input, Flatten, concatenate,LSTM,Dropout
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/yelp_data.csv')
data.head()

,business_id,user_id,stars,text
0,cPepkJeRMtHapc_b2Oe_dw,RreNy--tOmXMl1en0wiBOg,4.0,I was really between 3 and 4 stars for this on...
1,EtKSTHV5Qx_Q7Aur9o4kQQ,ZGjgfSvjQK886kiTzLwfLQ,5.0,"On a scale of one to things that are awesome, ..."
2,VJEzpfLs_Jnzgqh5A_FVTg,IKbjLnfBQtEyVzEu8CuOLg,4.0,It was my fiance's birthday and he decided he ...
3,TyOe_EcbyAWMmPgg_ILwHQ,enaHB1e956thdnafcHVAig,3.0,I've been to this location many times when I l...
4,oJ4ik-4PZe6gexxW-tSmsw,DBYhpb5hrAYgQjQaMhNYyQ,4.0,I love the Brewpub for their variety of dishes...


In [ ]:
ord_enc = OrdinalEncoder()
data['business_id'] =ord_enc.fit_transform(data['business_id'].values.reshape(-1, 1)).reshape(1,-1)[0]
data['user_id'] =ord_enc.fit_transform(data['user_id'].values.reshape(-1, 1)).reshape(1,-1)[0]

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import re
def remove_punc(line):
  line = re.sub('[0-9@]', '',line)
  line = re.sub('[#[]', '',line)
  line = re.sub(']', '',line)
  line = re.sub('[\'":,.?!&)(]', '',line)
  line = line.rstrip().lstrip()
  return line

data['text'] = data['text'].apply(lambda x : remove_punc(x))
data.text.values

array(['I was really between  and  stars for this one I LOVE the th street Naked Tchopstix so I was very excited to see this one which is closer to my house\nThe vibe is totally different as this is geared more to take out although they do have a decent size dining area You order at the counter and they deliver it\nMy daughter and I tried the sushi bowl You pick up a piece of paper and  select from a choice of proteins vegetables rice and toppings I like the fact that it is on paper and you dont have to guess what things are or have to shout over the counter to tell the server what you want The sushi bowl was good but not wonderful The choices particularly of the vegetables seemed a little too limiting and I felt I had more rice than other food items The sauces were tasty My husband ordered the Maui roll which is my favorite roll at the th street location and this just was not up that standard It was good but the quality and consistently was not as good as at th street I would go back 

In [ ]:
import nltk
from nltk.corpus import stopwords
stop_w = stopwords.words('english')

data['text'] = data['text'].apply(lambda x : ' '.join([w.lower() for w in x.split(' ') if w.lower() not in stop_w ]))
data['text'] = data['text'].apply(lambda x : ' '.join([t for t in x.split(' ') if len(t)>0]))
    
data.text.values

array(['really stars one love th street naked tchopstix excited see one closer house\nthe vibe totally different geared take although decent size dining area order counter deliver it\nmy daughter tried sushi bowl pick piece paper select choice proteins vegetables rice toppings like fact paper dont guess things shout counter tell server want sushi bowl good wonderful choices particularly vegetables seemed little limiting felt rice food items sauces tasty husband ordered maui roll favorite roll th street location standard good quality consistently good th street would go back though think lot people would happy sushi bowls want make comment low reviews people given place service people - sushi place - isnt mcdonalds- bit slower going order get five minutes even though called express healthy alternative fast food area worth wait oh try bubble tea strawberry quite yummy next time going try vanilla chai',
       'scale one things awesome place bomb \n\ni drawn promise duckfat frenchfries to

In [ ]:
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.layers import Dense, LSTM, Embedding,Bidirectional
import tensorflow
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
def mod():
  sent_inp = Input(shape=(80, 768))
  X = Bidirectional(CuDNNLSTM(512, return_sequences=True))(sent_inp)
  
  X = Bidirectional(CuDNNLSTM(100,return_sequences=True))(X)
  X = Dropout(0.2)(X)
  X = Bidirectional(CuDNNLSTM(200,return_sequences=True))(X)
  X = Dropout(0.2)(X)
  X = Bidirectional(CuDNNLSTM(100))(X)
  
  X = Dense(16,activation='relu',kernel_regularizer='l1')(X) 
  out = Dense(1,activation='sigmoid')(X)

 
  


  
  
  
  
  model = Model(inputs=sent_inp, 
                  outputs=out)
  
  return model


In [ ]:
model_sent = mod()

In [ ]:
from tensorflow.keras.models import load_model
model_sent = load_model('/content/drive/MyDrive/Yelp_model_sentiment.h5')

In [ ]:
!pip install --upgrade transformers

     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 84 kB 2.8 MB/s 
     |████████████████████████████████| 6.6 MB 33.4 MB/s 
     |████████████████████████████████| 596 kB 44.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer, TFBertModel,pipeline
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased")


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
texts = list(data.text.values)
for i in range(0,len(texts),10):
  try : 
    if (i%1000) == 0:
      print(i)
    encoded_input = tokenizer(texts[i:i+10], return_tensors='tf',max_length = 80,  # maximum length of a sentence  (TODO Figure the longest passage length)
          truncation = 'longest_first',padding='max_length'
        )
    output = model(encoded_input)
    for j in range(10):
        texts.append(np.copy(np.array(output['last_hidden_state'])[j]))
  except :
    pass

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000


In [ ]:
prd = model_sent.predict(texts)
prd

In [ ]:
'''texts = list(data.text.values)
i=0
pred = []
for text in texts :
  i+=1
  if i%1000==0:
    print(len(texts)-i)
  encoded_input = tokenizer([text], return_tensors='tf',max_length = 80,  # maximum length of a sentence  (TODO Figure the longest passage length)
      truncation = 'longest_first',padding='max_length'
    )
    
  output = model(encoded_input)
  prd = model_sent.predict(output['last_hidden_state'])[0]
  pred.append(1 if prd[0]>0.5 else 0)'''
  

In [ ]:
pred

In [ ]:
data['sentiment_score'] = pred

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,random_state=24,shuffle=True)

In [ ]:
f=0
for train_index, test_index in kf.split(data):
  print('fold number : ',f)
  f+=1
  train_data , test_data = data.iloc[train_index,:] , data.iloc[test_index,:]
  train_data.to_csv('/content/drive/MyDrive/Yelp_fold_with_sentiment'+str(f)+'_train.csv',index=False)
  test_data.to_csv('/content/drive/MyDrive/Yelp_fold_with_sentiment'+str(f) +'_test.csv',index=False)

In [ ]:
train_data